# rprev-ext multi-index demo
This notebook compares the **dev multi-index implementation** to the **CRAN rprev** baseline.

**What we test**
1. DEV K>1 run with fixed `index_dates` and shared parameters.
2. CRAN K=1 runs per index (baseline).
3. K=1 DEV vs CRAN equality check.
4. Targeted diagnostics to isolate where differences could come from.

**Observation (saved run)**
- K=1 DEV matches CRAN exactly for all horizons.
- Pre?registry simulated contribution matches CRAN.
- Counted prevalence matches CRAN.
- After the sim?ID fix, K>1 estimates equal counted + simulated contributions.


> **Repro note (commit checkout):**
> 
> This notebook documents a K>1 discrepancy that was traced to **sim?ID indexing** in the multi?index estimator.
> To reproduce the **buggy** behavior, check out the commit below and run the notebook unchanged.
> To see the **corrected** behavior, check out the fix commit.
> 
> - **Buggy behavior (K>1 underestimation):** `5d314634b9e4e85d30e496bd09f10f9c64599d4a`  
>   *Symptom:* the K>1 estimates were **too low** even though counted prevalence and pre?registry simulation matched CRAN.
>   *Cause:* `sim_agg$sim` was a **character** vector; indexing with it appended named elements and effectively **halved** the simulated contribution.
> 
> - **Fixed behavior (correct sim?ID indexing):** `d15eca9054ad410271ba85cdb53211fbef90c7c1`  
>   *Fix:* convert sim IDs to numeric before indexing so each bootstrap contribution is placed into the correct slot.
> 
> If you want to verify the bug, run the diagnostic section "Vector?fill bug illustration" ? it shows the raw vs fixed means side?by?side.


In [ ]:
# Setup: load local package + shared parameters
if (!requireNamespace('devtools', quietly = TRUE)) install.packages('devtools')
devtools::load_all("C:/Users/600972868/OneDrive - BT Plc/Desktop/Sajat Mappa/00 - BGE/Alkalmazott Matematika/Szakdolgozat/repo/rprev-ext")

library(lubridate)
library(survival)

data(prevsim)

# Shared parameters (use everywhere for consistency)
seed <- 123
N_boot <- 1000
num_years <- c(3, 5, 10, 13, 15)
index_dates <- as.Date(c('2012-01-07', '2013-01-07', '2014-01-07'))
registry_start <- min(prevsim$entrydate, na.rm = TRUE)

inc_formula <- entrydate ~ sex
surv_formula <- Surv(time, status) ~ age + sex
dist <- 'weibull'
population_size <- 1e6
death_column <- 'eventdate'
status_col <- 'status'



## 1) DEV multi-index run
We run the dev implementation once with fixed `index_dates` and shared parameters.

**Observation (saved run):** K>1 estimates are stable and no warnings appear.

**Why this matters:** this run defines the *single?population, multi?index* behavior described in the thesis.
If this output is inconsistent across horizons, it indicates a logic or indexing error, not RNG noise.


In [ ]:
set.seed(seed)
res <- prevalence(
  index_dates = index_dates,
  num_years_to_estimate = num_years,
  data = prevsim,
  inc_formula = inc_formula,
  surv_formula = surv_formula,
  dist = dist,
  population_size = population_size,
  death_column = death_column,
  registry_start_date = registry_start,
  N_boot = N_boot
)
res



### Summary output
**Observation (saved run):** summary prints index?by?horizon tables as expected.

We expect the summary to print one row per index date and one column per horizon.
If any horizon is missing or misaligned, it points to issues in estimate assembly or printing.


In [ ]:
summary(res)



## 2) CRAN rprev (single-index baseline)
We run CRAN **separately for each index date** (K=1 baseline).

**Observation (saved run):** CRAN prints standard output; used as comparison baseline.

**Baseline definition:** CRAN runs each index separately, so it re?simulates populations per index.
This is the reference behavior for K=1 in the original package.


In [ ]:
if (!requireNamespace('callr', quietly = TRUE)) install.packages('callr')
if (!requireNamespace('rprev', quietly = TRUE)) install.packages('rprev')

cran_outputs <- callr::r(function(index_dates, num_years, registry_start, seed, N_boot,
                                  inc_formula, surv_formula, dist, population_size, death_column) {
  library(rprev)
  library(lubridate)
  library(survival)
  data(prevsim, package = 'rprev')

  index_dates <- as.Date(index_dates)
  registry_start <- as.Date(registry_start)

  out <- lapply(index_dates, function(idx) {
    set.seed(seed)
    prevalence(
      index = idx,
      num_years_to_estimate = num_years,
      data = prevsim,
      inc_formula = inc_formula,
      surv_formula = surv_formula,
      dist = dist,
      population_size = population_size,
      death_column = death_column,
      registry_start_date = registry_start,
      N_boot = N_boot
    )
  })

  outputs <- lapply(seq_along(out), function(i) {
    idx <- index_dates[i]
    c(
      paste0('=== CRAN rprev index: ', idx, ' ==='),
      capture.output(print(out[[i]])),
      '',
      capture.output(summary(out[[i]])),
      ''
    )
  })
  unlist(outputs)
}, args = list(index_dates = index_dates,
              num_years = num_years,
              registry_start = registry_start,
              seed = seed,
              N_boot = N_boot,
              inc_formula = inc_formula,
              surv_formula = surv_formula,
              dist = dist,
              population_size = population_size,
              death_column = death_column))

cat(paste(cran_outputs, collapse = '
'))



## 3) K=1 comparison (DEV vs CRAN)
We compare K=1 results for the first index date.

**Observation (saved run):** all horizons match exactly (abs_diff = 0).

If this comparison fails, the issue is *not* multi?index logic; it is in the shared core (models, simulation, or counted logic).


In [ ]:
index_single <- index_dates[1]

extract_est_table <- function(res, years) {
  rows <- lapply(years, function(y) {
    item <- paste0('y', y)
    est <- res$estimates[[item]]
    abs_val <- est[['absolute.prevalence']]
    rate_name <- grep('^per[^.]*$', names(est), value=TRUE)
    rate_val <- if (length(rate_name) == 1) est[[rate_name]] else NA_real_
    data.frame(year = y, absolute = as.numeric(abs_val), rate = as.numeric(rate_val))
  })
  do.call(rbind, rows)
}



In [ ]:
# K=1 run using local (dev) package
set.seed(seed)
res_dev_k1 <- prevalence(
  index_dates = index_single,
  num_years_to_estimate = num_years,
  data = prevsim,
  inc_formula = inc_formula,
  surv_formula = surv_formula,
  dist = dist,
  population_size = population_size,
  death_column = death_column,
  registry_start_date = registry_start,
  N_boot = N_boot
)
dev_table <- extract_est_table(res_dev_k1, num_years)
dev_table



In [ ]:
# K=1 run using CRAN rprev in a separate R session
if (!requireNamespace('callr', quietly = TRUE)) install.packages('callr')
if (!requireNamespace('rprev', quietly = TRUE)) install.packages('rprev')

cran_table <- callr::r(function(idx, years, seed, registry_start, N_boot, pop_size,
                                inc_formula, surv_formula, dist, death_column) {
  library(rprev)
  library(lubridate)
  library(survival)
  data(prevsim, package = 'rprev')

  set.seed(seed)
  res <- prevalence(
    index = idx,
    num_years_to_estimate = years,
    data = prevsim,
    inc_formula = inc_formula,
    surv_formula = surv_formula,
    dist = dist,
    population_size = pop_size,
    death_column = death_column,
    registry_start_date = registry_start,
    N_boot = N_boot
  )

  extract_est_table <- function(res, years) {
    rows <- lapply(years, function(y) {
      item <- paste0('y', y)
      est <- res$estimates[[item]]
      abs_val <- est[['absolute.prevalence']]
      rate_name <- grep('^per[^.]*$', names(est), value=TRUE)
      rate_val <- if (length(rate_name) == 1) est[[rate_name]] else NA_real_
      data.frame(year = y, absolute = as.numeric(abs_val), rate = as.numeric(rate_val))
    })
    do.call(rbind, rows)
  }

  extract_est_table(res, years)
}, args = list(idx = index_single,
              years = num_years,
              seed = seed,
              registry_start = registry_start,
              N_boot = N_boot,
              pop_size = population_size,
              inc_formula = inc_formula,
              surv_formula = surv_formula,
              dist = dist,
              death_column = death_column))
cran_table



In [ ]:
# Comparison table
compare_table <- merge(dev_table, cran_table, by = 'year', suffixes = c('_dev', '_cran'))
compare_table$abs_diff <- compare_table$absolute_dev - compare_table$absolute_cran
compare_table$rate_diff <- compare_table$rate_dev - compare_table$rate_cran
compare_table



## 4) Prefix-effect demo (K=1 vs K>1 with expanding horizons)
We check whether adding extra horizons changes the 13y estimate.

**Observation (saved run):** 13y is consistent across K settings (differences ? 0).

This check ensures that adding extra horizons does **not** shift earlier horizons.
If it does, the aggregation is order?dependent, which would be a bug.


In [ ]:
# Prefix-effect demo: K=1 vs K>1 with expanding max horizon
set.seed(seed)
years_13 <- c(13)
years_10_13 <- c(10, 13)
years_10_13_15 <- c(10, 13, 15)

res_k1_13 <- prevalence(
  index_dates = index_dates,
  num_years_to_estimate = years_13,
  data = prevsim,
  inc_formula = inc_formula,
  surv_formula = surv_formula,
  dist = dist,
  population_size = population_size,
  death_column = death_column,
  registry_start_date = registry_start,
  N_boot = N_boot
)

set.seed(seed)
res_k10_13 <- prevalence(
  index_dates = index_dates,
  num_years_to_estimate = years_10_13,
  data = prevsim,
  inc_formula = inc_formula,
  surv_formula = surv_formula,
  dist = dist,
  population_size = population_size,
  death_column = death_column,
  registry_start_date = registry_start,
  N_boot = N_boot
)

set.seed(seed)
res_k10_13_15 <- prevalence(
  index_dates = index_dates,
  num_years_to_estimate = years_10_13_15,
  data = prevsim,
  inc_formula = inc_formula,
  surv_formula = surv_formula,
  dist = dist,
  population_size = population_size,
  death_column = death_column,
  registry_start_date = registry_start,
  N_boot = N_boot
)

# Extract 13y estimates for comparison (first index date)
get_y <- function(res, y) {
  item <- paste0('y', y)
  as.numeric(res$estimates[[item]]$absolute.prevalence)[1]
}

data.frame(
  case = c('K=1 (13y only)', 'K>1 (10+13)', 'K>1 (10+13+15)'),
  y13 = c(get_y(res_k1_13, 13), get_y(res_k10_13, 13), get_y(res_k10_13_15, 13))
)



## 5) Diagnostics: aggregation shape and completeness
We verify that simulated aggregation covers all (sim ? index ? year) rows.

**Observation (saved run):** expected rows match exactly, no missing sims.

We verify that (sim ? index ? year) is fully populated and there are no missing sims.
Missing rows or NA contributions would bias means downward.


In [ ]:
# Diagnostics: inspect simulated aggregation shapes and missing sims
library(data.table)

sim_dt <- as.data.table(res$simulated)
years_diag <- num_years
idx_diag <- index_dates
reg_start_diag <- registry_start

sim_prev_counts <- build_prev_counts_multiindex(sim_dt, idx_diag, years_diag)
sim_prev_counts_pre <- build_prev_counts_multiindex(sim_dt[incident_date < reg_start_diag], idx_diag, years_diag)

n_sims <- length(unique(sim_dt$sim))
expected_rows <- n_sims * length(idx_diag) * length(years_diag)
data.frame(
  n_sims = n_sims,
  expected_rows = expected_rows,
  nrow_total = nrow(sim_prev_counts),
  nrow_pre = nrow(sim_prev_counts_pre)
)

# Build sim_agg in the same way prevalence() does
total <- as.data.frame(sim_prev_counts)
total$index_dates <- idx_diag[total$k]
total$contrib_total <- total$prev_count
total <- total[, c('sim','index_dates','year','contrib_total')]

pre <- as.data.frame(sim_prev_counts_pre)
pre$index_dates <- idx_diag[pre$k]
pre$contrib_pre_registry <- pre$prev_count
pre <- pre[, c('sim','index_dates','year','contrib_pre_registry')]

sim_agg_dbg <- merge(total, pre, by = c('sim','index_dates','year'), all = TRUE)
data.frame(
  nrow_sim_agg = nrow(sim_agg_dbg),
  expected_rows = expected_rows,
  na_contrib_total = sum(is.na(sim_agg_dbg$contrib_total)),
  na_contrib_pre = sum(is.na(sim_agg_dbg$contrib_pre_registry))
)

# Check if any sim IDs are missing entirely
missing_sims <- setdiff(seq_len(max(sim_dt$sim)), unique(sim_dt$sim))
missing_sims



### 5.1 Year?set invariance
We verify that the 13y aggregation is identical whether computed alone or with other horizons.

**Observation (saved run):** max_abs_diff = 0.

If `max_abs_diff` is non?zero here, the aggregation is inconsistent and needs fixing.


In [ ]:
# Diagnostic: does build_prev_counts_multiindex depend on the years set?
library(data.table)

sim_dt2 <- as.data.table(res$simulated)
idx_diag2 <- index_dates
years_only_13 <- c(13)
years_10_13_15 <- c(10, 13, 15)

pc_13_only <- build_prev_counts_multiindex(sim_dt2, idx_diag2, years_only_13)
pc_10_13_15 <- build_prev_counts_multiindex(sim_dt2, idx_diag2, years_10_13_15)

# Extract 13-year rows from both and compare
pc_13_only <- pc_13_only[year == 13]
pc_13_from_all <- pc_10_13_15[year == 13]

setkey(pc_13_only, sim, k)
setkey(pc_13_from_all, sim, k)
cmp <- pc_13_only[pc_13_from_all]  # join by sim,k
cmp[, diff := prev_count - i.prev_count]

data.frame(
  n_rows_only = nrow(pc_13_only),
  n_rows_from_all = nrow(pc_13_from_all),
  max_abs_diff = if (nrow(cmp) > 0) max(abs(cmp$diff), na.rm = TRUE) else NA_real_,
  n_diff = if (nrow(cmp) > 0) sum(cmp$diff != 0, na.rm = TRUE) else 0
)



### 5.2 Warning trap (optional)
We force warnings to error to catch any recycling issues.

**Observation (saved run):** no warnings were raised.

We intentionally fail hard on warnings to catch hidden recycling or indexing issues.
This previously caught the data.table recycling bug.


In [ ]:
# Debug: turn warnings into errors to locate recycling source
options(warn = 2)
tryCatch({
  set.seed(seed)
  res_dbg <- prevalence(
    index_dates = index_dates,
    num_years_to_estimate = num_years,
    data = prevsim,
    inc_formula = inc_formula,
    surv_formula = surv_formula,
    dist = dist,
    population_size = population_size,
    death_column = death_column,
    registry_start_date = registry_start,
    N_boot = min(100, N_boot)
  )
  res_dbg
}, warning = function(w) {
  cat('WARNING AS ERROR:
')
  print(w)
  cat('Traceback:
')
  traceback(2)
}, error = function(e) {
  cat('ERROR:
')
  print(e)
  cat('Traceback:
')
  traceback(2)
})
options(warn = 0)



## 6) Pre?registry simulated contribution (DEV vs CRAN)
We compare the **simulated pre?registry mean and SD** for the same index and horizon.

**Observation (saved run):** dev_mean ? cran_mean.

If DEV and CRAN differ here, the issue is in simulated pre?registry contributions.
If they match, the discrepancy must be elsewhere (combination logic).


In [ ]:
# Compare pre-registry simulated contributions: DEV vs CRAN (same index + year)
library(data.table)

# --- DEV side: use sim_agg built from res$simulated ---
sim_dt_dev <- as.data.table(res$simulated)
sim_prev_total_dev <- build_prev_counts_multiindex(sim_dt_dev, index_dates, num_years)
sim_prev_pre_dev <- build_prev_counts_multiindex(sim_dt_dev[incident_date < registry_start], index_dates, num_years)

total_dev <- as.data.frame(sim_prev_total_dev)
total_dev$index_dates <- index_dates[total_dev$k]
total_dev$contrib_total <- total_dev$prev_count
total_dev <- total_dev[, c('sim','index_dates','year','contrib_total')]

pre_dev <- as.data.frame(sim_prev_pre_dev)
pre_dev$index_dates <- index_dates[pre_dev$k]
pre_dev$contrib_pre_registry <- pre_dev$prev_count
pre_dev <- pre_dev[, c('sim','index_dates','year','contrib_pre_registry')]

sim_agg_dev <- merge(total_dev, pre_dev, by=c('sim','index_dates','year'), all=TRUE)

# Pick one index + year to compare (edit here if needed)
idx_pick <- index_dates[1]
year_pick <- max(num_years)

dev_vec <- sim_agg_dev$contrib_pre_registry[sim_agg_dev$index_dates == idx_pick & sim_agg_dev$year == year_pick]

# --- CRAN side: run and extract pre-registry contribs from res$simulated ---
if (!requireNamespace('callr', quietly = TRUE)) install.packages('callr')
if (!requireNamespace('rprev', quietly = TRUE)) install.packages('rprev')

cran_vec <- callr::r(function(idx, year_pick, registry_start, N_boot, seed,
                             inc_formula, surv_formula, dist, population_size, death_column) {
  library(rprev)
  library(lubridate)
  library(data.table)
  data(prevsim, package = 'rprev')

  set.seed(seed)
  res_c <- prevalence(
    index = idx,
    num_years_to_estimate = year_pick,
    data = prevsim,
    inc_formula = inc_formula,
    surv_formula = surv_formula,
    dist = dist,
    population_size = population_size,
    death_column = death_column,
    registry_start_date = registry_start,
    N_boot = N_boot
  )

  sim_dt <- as.data.table(res_c$simulated)
  # rebuild prev_* columns for the chosen year
  col_name <- paste0('prev_', year_pick, 'yr')
  sim_vec <- sim_dt[incident_date < registry_start, sum(get(col_name)), by=sim][[2]]
  sim_vec
}, args = list(idx = idx_pick,
              year_pick = year_pick,
              registry_start = registry_start,
              N_boot = N_boot,
              seed = seed,
              inc_formula = inc_formula,
              surv_formula = surv_formula,
              dist = dist,
              population_size = population_size,
              death_column = death_column))

data.frame(
  index_date = as.character(idx_pick),
  year = year_pick,
  dev_mean = mean(dev_vec),
  dev_sd = sd(dev_vec),
  cran_mean = mean(cran_vec),
  cran_sd = sd(cran_vec)
)



## 7) Counted prevalence match (DEV vs CRAN)
We verify the counted component is identical.

**Observation (saved run):** counted values match exactly.

This isolates the counted component.
If this fails, the discrepancy is in registry?only counting, not simulation.


In [ ]:
# Compare counted_prevalence piece: DEV (K>1) vs CRAN (K=1)
library(lubridate)

idx_pick <- index_dates[1]
year_pick <- max(num_years)
initial_date <- idx_pick - lubridate::years(year_pick)
start_k <- max(initial_date, registry_start)

# DEV counted_prevalence directly (uses local function)
counted_dev <- counted_prevalence(eventdate ~ entrydate, idx_pick, prevsim, start_k, status_col = status_col)

# CRAN counted_prevalence via separate session
cran_counted <- callr::r(function(idx, start_k) {
  library(rprev)
  data(prevsim, package = 'rprev')
  rprev:::counted_prevalence(eventdate ~ entrydate, idx, prevsim, start_k, status_col = 'status')
}, args = list(idx = idx_pick, start_k = start_k))

data.frame(
  index_date = as.character(idx_pick),
  year = year_pick,
  start_date = as.character(start_k),
  counted_dev = counted_dev,
  counted_cran = cran_counted
)



## 8) DEV K>1 estimate decomposition
We check whether `counted + sim_mean` equals the reported estimate.

**Observation (saved run):** counted + sim_mean equals estimate_reported.

This check proves whether the final estimate equals the sum of its parts.
If not, the estimator is dropping or mis?indexing simulated contributions.


In [ ]:
# Decompose DEV K>1 estimate into counted + sim for one index/year
library(data.table)

idx_pick <- index_dates[1]
year_pick <- max(num_years)

# Rebuild sim_agg from current res (K>1 run)
sim_dt_dev <- as.data.table(res$simulated)
sim_prev_total_dev <- build_prev_counts_multiindex(sim_dt_dev, index_dates, num_years)
sim_prev_pre_dev <- build_prev_counts_multiindex(sim_dt_dev[incident_date < registry_start], index_dates, num_years)

total_dev <- as.data.frame(sim_prev_total_dev)
total_dev$index_dates <- index_dates[total_dev$k]
total_dev$contrib_total <- total_dev$prev_count
total_dev <- total_dev[, c('sim','index_dates','year','contrib_total')]

pre_dev <- as.data.frame(sim_prev_pre_dev)
pre_dev$index_dates <- index_dates[pre_dev$k]
pre_dev$contrib_pre_registry <- pre_dev$prev_count
pre_dev <- pre_dev[, c('sim','index_dates','year','contrib_pre_registry')]

sim_agg_dev <- merge(total_dev, pre_dev, by=c('sim','index_dates','year'), all=TRUE)

# Counted part
initial_date <- idx_pick - lubridate::years(year_pick)
need_sim <- initial_date < registry_start
start_k <- max(initial_date, registry_start)
count_prev <- counted_prevalence(eventdate ~ entrydate, idx_pick, prevsim, start_k, status_col = status_col)

# Sim part (pre-registry if needed)
if (need_sim) {
  sim_sub <- sim_agg_dev[sim_agg_dev$index_dates == idx_pick & sim_agg_dev$year == year_pick, ]
  sim_contribs <- sim_sub$contrib_pre_registry
  sim_mean <- mean(sim_contribs)
} else {
  sim_mean <- 0
}

# Actual estimate from res$estimates
est_val <- res$estimates[[paste0('y', year_pick)]]$absolute.prevalence[1]

data.frame(
  index_date = as.character(idx_pick),
  year = year_pick,
  need_sim = need_sim,
  counted = count_prev,
  sim_mean = sim_mean,
  counted_plus_sim = count_prev + sim_mean,
  estimate_reported = est_val
)



## 9) sim IDs and type
We check that `sim` IDs are contiguous and inspect their type.

**Observation (saved run):** IDs are 1..N_boot but stored as character.

Character sim IDs are dangerous if used as numeric indices.
We expect IDs to be numeric 1..N_boot.


In [ ]:
# Inspect sim IDs in sim_agg_dev (should be 1..N_boot)
class(sim_agg_dev$sim)
range(sim_agg_dev$sim)
head(sort(unique(sim_agg_dev$sim)))
tail(sort(unique(sim_agg_dev$sim)))



## 10) Vector?fill bug illustration (now fixed)
We show how character sim IDs can halve the mean if used for indexing.

**Observation (saved run):** raw mean_filled was ~half; fixed indexing restores equality.

This shows exactly why the bug caused a large downward bias.
The raw mean is about half because indexing by character names appends instead of fills.


In [ ]:
# Compare sim_agg mean vs vector-fill (raw vs fixed)
idx_pick <- index_dates[1]
year_pick <- max(num_years)

sub <- sim_agg_dev[sim_agg_dev$index_dates == idx_pick & sim_agg_dev$year == year_pick, ]
mean_direct <- mean(sub$contrib_pre_registry)

# Raw fill (character sim IDs)
vec_raw <- rep(0, N_boot)
vec_raw[sub$sim] <- sub$contrib_pre_registry
mean_filled_raw <- mean(vec_raw)

# Fixed fill (numeric sim IDs)
sim_ids <- suppressWarnings(as.integer(as.character(sub$sim)))
vec_fixed <- rep(0, N_boot)
vec_fixed[sim_ids] <- sub$contrib_pre_registry
mean_filled_fixed <- mean(vec_fixed)

data.frame(
  index_date = as.character(idx_pick),
  year = year_pick,
  mean_direct = mean_direct,
  mean_filled_raw = mean_filled_raw,
  mean_filled_fixed = mean_filled_fixed,
  N_boot = N_boot
)

